In [2]:
import json
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import nltk


In [3]:
import json
with open('data.json') as json_data:
    intents = json.load(json_data)
    

In [4]:
lines = []
with open('./train_sentences/train_emotion.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        str,emo = line.split(';')
        if 'sadness' in emo: 
                if str not in intents['intents'][3]['patterns'] and len(intents['intents'][3]['patterns']) <= 400:
                        intents['intents'][3]['patterns'].append(str)
        if 'joy' in emo: 
                if str not in intents['intents'][4]['patterns']and len(intents['intents'][4]['patterns']) <= 400:
                        intents['intents'][4]['patterns'].append(str)
        if 'anger' in emo: 
                if str not in intents['intents'][5]['patterns']and len(intents['intents'][5]['patterns']) <= 400:
                        intents['intents'][5]['patterns'].append(str)
        if 'fear' in emo: 
                if str not in intents['intents'][6]['patterns']and len(intents['intents'][6]['patterns']) <= 400:
                        intents['intents'][6]['patterns'].append(str)


In [5]:
lines = []
with open('./train_sentences/train_greet.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][0]['patterns']:
            intents['intents'][0]['patterns'].append(line)

lines = []
with open('./train_sentences/train_bye.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][1]['patterns']:
            intents['intents'][1]['patterns'].append(line)

lines = []
with open('./train_sentences/train_thanks.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][2]['patterns']:
            intents['intents'][2]['patterns'].append(line)


with open('data2.json', 'w') as outfile:
        json.dump(intents, outfile)


In [6]:
with open('data2.json') as json_data:
    intents = json.load(json_data)

In [7]:
training_sentences = []
training_labels = []
labels = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [8]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

training_labels

array([3, 3, 3, ..., 1, 1, 1])

In [9]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<00V>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen = max_len)


In [10]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation = 'relu'))
model.add(Dense(16,activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 7)                 119       
                                                                 
Total params: 16,663
Trainable params: 16,663
Non-trainable params: 0
____________________________________________________

In [11]:
history = model.fit(padded_sequences, np.array(training_labels), epochs = 100)

Epoch 1/500
56/56 [==============================] - 0s 1ms/step - loss: 1.9116 - accuracy: 0.2257
Epoch 2/500
56/56 [==============================] - 0s 1ms/step - loss: 1.7940 - accuracy: 0.2252
Epoch 3/500
56/56 [==============================] - 0s 1ms/step - loss: 1.6299 - accuracy: 0.2263
Epoch 4/500
56/56 [==============================] - 0s 2ms/step - loss: 1.5002 - accuracy: 0.3144
Epoch 5/500
56/56 [==============================] - 0s 2ms/step - loss: 1.4411 - accuracy: 0.3908
Epoch 6/500
56/56 [==============================] - 0s 2ms/step - loss: 1.3786 - accuracy: 0.4879
Epoch 7/500
56/56 [==============================] - 0s 2ms/step - loss: 1.3037 - accuracy: 0.5020
Epoch 8/500
56/56 [==============================] - 0s 2ms/step - loss: 1.2215 - accuracy: 0.5508
Epoch 9/500
56/56 [==============================] - 0s 2ms/step - loss: 1.1364 - accuracy: 0.5834
Epoch 10/500
56/56 [==============================] - 0s 2ms/step - loss: 1.0502 - accuracy: 0.5980
Epoch 11/

In [12]:
model.save("chat_model")

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
